In [1]:
import pandas as pd
from Sentiment.sentiment_script import vader_sentiment, split_into_sentences
from tqdm import tqdm
from NER_and_ED.NER_ED_script import perform_ner_for_sentences
from Topics.bratopic_script import perform_bratopic
dict_name = "../../NER_and_ED/dict_ukraine.csv"

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\lukas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
# Load data
df = pd.read_csv("../ukraine_textcontain_after_new_preprocessed.csv")
df

,Unnamed: 0,article_title,author,published_time,article_text,article_category_one,article_category_two,picture_description,author_title,author_description
0,0,US can no longer be perfunctory toward Guterre...,Global Times,2022-08-22,"Since this year, the United Nations (UN) has r...",OPINION,EDITORIAL,United Nations Secretary-General Antonio Guter...,Author details not found,Author details not found
1,1,Healthy and stable China-S.Korea ties depend o...,Wang Junsheng,2022-08-23,Wednesday marks the 30th anniversary of the es...,OPINION,VIEWPOINT,Illustration: Chen Xia/GT,Author details not found,Author details not found
2,2,"China’s power in the world, economically and p...",Global Times,2022-08-23,"Editor's Note: For Chinese people, the past de...",OPINION,VIEWPOINT,"Tower Bridge, London, the UK Photo:VCG",Author details not found,Author details not found
3,3,Europe faces ‘coldest’ winter as energy crisis...,Qi Xijia,2022-08-23,A deepening energy crisis in Europe could put ...,SOURCE,ECONOMY,A coal-fired power plant operated by German en...,Author details not found,Author details not found
4,4,Severe drought in Europe ‘further expanding an...,AFP,2022-08-23,A severe drought hitting swathes of Europe is ...,WORLD,EUROPE,Picture description not found,Author details not found,Author details not found
...,...,...,...,...,...,...,...,...,...,...
484,485,US policy of ‘technological apartheid’ could l...,William Jones,2022-06-06,A New York Times article earlier this month do...,OPINION,VIEWPOINT,Illustration: Tang Tengfei/Global Times,Author details not found,Author details not found
485,486,US behind global grain crises: Chinese FM spok...,Global Times,2022-06-07,Zhao Lijian Photo: VCG Each global grain cris...,SOURCE,ECONOMY,Picture description not found,Author details not found,Author details not found
486,487,"‘No longer US’ backyard,’ Latin America sends ...",GT staff reporters,2022-06-07,Mexico's President Andres Manuel Lopez Obrador...,WORLD,AMERICAS,Picture description not found,Author details not found,Author details not found
487,488,Chinese envoy warns against providing weapons ...,Xinhua,2022-06-07,A Chinese envoy on Monday warned against const...,CHINA,DIPLOMACY,Picture description not found,Author details not found,Author details not found


In [ ]:
article_text = []
article_id = []
dates = []

for i in range(len(df)):
    article_text.append(str(df['article_text'][i]))
    date = str(df['published_time'][i])
    title = str(df['article_text'][i]).replace(" ", "_")
    id =  str(df['Unnamed: 0'][i])
    article_id.append(id)

In [ ]:
print(len(article_id))
#count unique article_id
print(len(set(article_id)))
article_id

In [ ]:
senteces_all = []
sentences_id_all = []
senteces_date_all = []

for article, id, date in tqdm(zip(article_text, article_id, df['published_time'])):
    sentences = split_into_sentences(article)
    sentences_id = []
    for i in range(len(sentences)):
        #sentence number 3digits with 0 padding
        sentences_id.append(str(id).zfill(3) + "-" + str(i).zfill(3))
        senteces_date_all.append(date)
    senteces_all += sentences
    sentences_id_all += sentences_id

In [ ]:
articles_id = [x.split("-")[0] for x in sentences_id_all]
sentences_df = pd.DataFrame({'sentence_id': sentences_id_all, 'article_id': articles_id,  'text': senteces_all, 'published_time': senteces_date_all})
sentences_df

In [ ]:
#sentiment
sentences_df['sentiment'] = sentences_df['text'].apply(vader_sentiment)
sentences_df

In [ ]:
ner_df = perform_ner_for_sentences(sentences_df, dict_name)
ner_df

In [36]:
#ner_df.to_csv("senteces_ukraine_after.csv", index=False)
ner_df = pd.read_csv("senteces_ukraine_after.csv")
ner_df

,Unnamed: 0,sentence_id,article_id,text,sentiment,NER,published_time,topic_id
0,0,000-000,0,"Since this year, the UN (UN) has repeatedly so...",negative,"{'UN': ('ORG', 2)}",2022-08-22,11
1,12,000-012,0,"Apart from the food issue, many hegemonism and...",negative,"{'US': ('LOC', 1)}",2022-08-22,11
2,14,000-014,0,"In addition, on one side, it is the scene of p...",negative,"{'Wall Street': ('LOC', 1)}",2022-08-22,11
3,15,000-015,0,"The four companies known as the four ""largest ...",neutral,"{'US': ('LOC', 1)}",2022-08-22,11
4,16,000-016,0,"According to US media reports, high food price...",positive,"{'US': ('LOC', 2)}",2022-08-22,11
...,...,...,...,...,...,...,...,...
16811,12781,374-003,374,"Last week, Turkey said documents it received f...",neutral,"{'Turkey': ('LOC', 2), 'Sweden': ('LOC', 1), '...",2022-06-21,35
16812,12783,374-005,374,Ibrahim Kalin said Ankara was expecting Sweden...,positive,"{'Ibrahim Kalin': ('PER', 1), 'Ankara': ('LOC'...",2022-06-21,35
16813,12784,374-006,374,"Any progress on the Nordic membership bids ""no...",positive,"{'Nordic': ('MISC', 1)}",2022-06-21,35
16814,12785,374-007,374,The talks in Brussels with officials from Swed...,positive,"{'Brussels': ('LOC', 1), 'Sweden': ('LOC', 1),...",2022-06-21,35


In [37]:
#topics
topics, probs, topic_detailes_df, topic_model = perform_bratopic(ner_df)

16816


Batches:   0%|          | 0/526 [00:00<?, ?it/s]

2025-01-03 23:00:26,587 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/526 [00:00<?, ?it/s]

2025-01-03 23:02:18,439 - BERTopic - Embedding - Completed ✓
2025-01-03 23:02:18,440 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-01-03 23:02:21,923 - BERTopic - Dimensionality - Completed ✓
2025-01-03 23:02:21,925 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-01-03 23:02:23,860 - BERTopic - Cluster - Completed ✓
2025-01-03 23:02:23,864 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-01-03 23:02:26,094 - BERTopic - Representation - Completed ✓


In [43]:
topic_detailes_df

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4912,-1_china_russia_countries_global,"[china, russia, countries, global, economic, a...",[The Ukraine crisis triggered by geopolitics a...
1,0,2084,0_world_countries_democracy_security,"[world, countries, democracy, security, politi...",[In the face of the most serious systemic cris...
2,1,921,1_china_relations_sovereignty_peace,"[china, relations, sovereignty, peace, countri...",[Most peace loving countries in the region res...
3,2,884,2_xinjiang_training_institute_vocational,"[xinjiang, training, institute, vocational, ar...",[Xinjiang Uyghur Autonomous Region Commerce Ec...
4,3,692,3_taiwan_straits_mainland_independence,"[taiwan, straits, mainland, independence, prin...",[ The China People s Liberation Army PLA condu...
5,4,607,4_ukraine_russia_kiev_weapons,"[ukraine, russia, kiev, weapons, zelensky, vol...",[The administration of U S President Joe Biden...
6,5,452,5_inflation_recession_fed_monetary,"[inflation, recession, fed, monetary, rates, c...",[US inflation has recently hit a year high of ...
7,6,434,6_brics_bri_countries_cooperation,"[brics, bri, countries, cooperation, emerging,...",[In response to media questions over China s v...
8,7,388,7_japan_kishida_constitution_military,"[japan, kishida, constitution, military, tokyo...",[On Friday Japan Prime Minister Fumio Kishida ...
9,8,310,8_nato_expansion_soviet_warsaw,"[nato, expansion, soviet, warsaw, alliance, yu...",[After the Cold War even though there were peo...


In [38]:
topics_df = ner_df
topics_df['topic_id'] = topics
topics_df

,Unnamed: 0,sentence_id,article_id,text,sentiment,NER,published_time,topic_id
0,0,000-000,0,"Since this year, the UN (UN) has repeatedly so...",negative,"{'UN': ('ORG', 2)}",2022-08-22,10
1,12,000-012,0,"Apart from the food issue, many hegemonism and...",negative,"{'US': ('LOC', 1)}",2022-08-22,10
2,14,000-014,0,"In addition, on one side, it is the scene of p...",negative,"{'Wall Street': ('LOC', 1)}",2022-08-22,10
3,15,000-015,0,"The four companies known as the four ""largest ...",neutral,"{'US': ('LOC', 1)}",2022-08-22,10
4,16,000-016,0,"According to US media reports, high food price...",positive,"{'US': ('LOC', 2)}",2022-08-22,10
...,...,...,...,...,...,...,...,...
16811,12781,374-003,374,"Last week, Turkey said documents it received f...",neutral,"{'Turkey': ('LOC', 2), 'Sweden': ('LOC', 1), '...",2022-06-21,33
16812,12783,374-005,374,Ibrahim Kalin said Ankara was expecting Sweden...,positive,"{'Ibrahim Kalin': ('PER', 1), 'Ankara': ('LOC'...",2022-06-21,33
16813,12784,374-006,374,"Any progress on the Nordic membership bids ""no...",positive,"{'Nordic': ('MISC', 1)}",2022-06-21,33
16814,12785,374-007,374,The talks in Brussels with officials from Swed...,positive,"{'Brussels': ('LOC', 1), 'Sweden': ('LOC', 1),...",2022-06-21,33


In [40]:
#add topic represnetation to the ner_df
topics_df = topics_df.merge(topic_detailes_df[['Topic', 'Representation']], left_on='topic_id', right_on='Topic')
topics_df = topics_df.drop(columns=['Topic'], inplace=False)
#rename column
topics_df = topics_df.rename(columns={"Representation": "topic_representation"})
topics_df.sort_values(by=['sentence_id'], inplace=True)
topics_df

,Unnamed: 0,sentence_id,article_id,text,sentiment,NER,published_time,topic_id,topic_representation,topic_representation
0,0,000-000,0,"Since this year, the UN (UN) has repeatedly so...",negative,"{'UN': ('ORG', 2)}",2022-08-22,10,"[grain, prices, wheat, agricultural, hunger, f...","[grain, prices, wheat, agricultural, hunger, f..."
5641,1,000-001,0,UN Secretary-General Antonio Guterres on Satur...,neutral,"{'UN': ('ORG', 1), 'Antonio Guterres': ('PER',...",2022-08-22,18,"[ukraine, turkey, food, istanbul, ships, guter...","[ukraine, turkey, food, istanbul, ships, guter..."
296,2,000-002,0,This is a major appeal made by UN Secretary-Ge...,positive,"{'UN': ('ORG', 1)}",2022-08-22,-1,"[china, russia, countries, global, economic, a...","[china, russia, countries, global, economic, a..."
297,3,000-003,0,Guterres had clear target when he made these r...,positive,"{'Guterres': ('PER', 1)}",2022-08-22,-1,"[china, russia, countries, global, economic, a...","[china, russia, countries, global, economic, a..."
4169,4,000-004,0,"At present, it is no secret who is the biggest...",negative,"{'Russia': ('LOC', 1)}",2022-08-22,-1,"[china, russia, countries, global, economic, a...","[china, russia, countries, global, economic, a..."
...,...,...,...,...,...,...,...,...,...,...
4168,16811,489-001,489,The agreement was reached during a meeting of ...,positive,"{'Vyacheslav Demchenko': ('PER', 1), 'Igor Tal...",2022-06-07,-1,"[china, russia, countries, global, economic, a...","[china, russia, countries, global, economic, a..."
14876,16812,489-002,489,The two countries agreed to work out mechanism...,positive,{},2022-06-07,26,"[freight, trains, port, eu, customs, mongolia,...","[freight, trains, port, eu, customs, mongolia,..."
14877,16813,489-003,489,"In particular, the parties agreed to optimize ...",positive,{},2022-06-07,26,"[freight, trains, port, eu, customs, mongolia,...","[freight, trains, port, eu, customs, mongolia,..."
10814,16814,489-004,489,"Besides, Ukraine and Moldova decided to consid...",neutral,"{'Ukraine': ('LOC', 2), 'Moldova': ('LOC', 2)}",2022-06-07,4,"[ukraine, russia, kiev, weapons, zelensky, vol...","[ukraine, russia, kiev, weapons, zelensky, vol..."


In [42]:
topics_df.to_csv("senteces_ukraine_after.csv", index=False)